<a href="https://colab.research.google.com/github/audreynk/KomputasiIntelegensia/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Audrey Naomi Karenina\
NPM: 2206048796\
Model: [Hugging Face](https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis)

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [ ]:
import torch

In [ ]:
# Test sentence
sentence = input("Enter a sentence for sentiment analysis: ")
inputs = tokenizer(sentence, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Sentiment prediction
labels = ['negative', 'neutral', 'positive']
print(f"Sentiment: {labels[predictions.argmax()]} (Confidence: {predictions.max().item():.2f})")

Enter a sentence for sentiment analysis: You are such a bitch
Sentiment: neutral (Confidence: 0.99)


In [ ]:
from datasets import Dataset

# Define synthetic dataset
data = {
    "text": ["The stock soared today!", "This product is disappointing.", "Market growth is stable.", "This is a risky investment."],
    "label": [2, 0, 1, 0]  # 2 = positive, 1 = neutral, 0 = negative
}
dataset = Dataset.from_dict(data)

def evaluate_model(dataset, model):
    correct = 0
    for sample in dataset:
        inputs = tokenizer(sample['text'], return_tensors="pt")
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        if prediction == sample['label']:
            correct += 1
    accuracy = correct / len(dataset)
    return accuracy

print("Initial Model Accuracy:", evaluate_model(dataset, model))

Initial Model Accuracy: 0.75


In [ ]:
from transformers import AutoModel
import torch.nn as nn

class AttentionTransformer(nn.Module):
    def __init__(self, model_name):
        super(AttentionTransformer, self).__init__()
        # Load DistilRoBERTa base model
        self.base_model = AutoModel.from_pretrained(model_name)
        self.attention = nn.MultiheadAttention(embed_dim=768, num_heads=8)
        self.classifier = nn.Linear(768, 3)  # 3 classes: negative, neutral, positive

    def forward(self, input_ids, attention_mask):
        # Get hidden states from DistilRoBERTa
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

        # Apply attention
        attn_output, _ = self.attention(hidden_states, hidden_states, hidden_states)

        # Classification (take the [CLS] token output, which is the first token)
        logits = self.classifier(attn_output[:, 0, :])
        return logits

In [ ]:
# Initialize the modified model
attention_model = AttentionTransformer("distilroberta-base")

def evaluate_attention_model(dataset, model):
    correct = 0
    for sample in dataset:
        inputs = tokenizer(sample['text'], return_tensors="pt")
        logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
        prediction = torch.argmax(logits, dim=1).item()
        if prediction == sample['label']:
            correct += 1
    accuracy = correct / len(dataset)
    return accuracy

print("Model Accuracy with Attention:", evaluate_attention_model(dataset, attention_model))

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Model Accuracy with Attention: 0.25
